# UKB -- this notebook creates the df you will need to run the table 3 analysis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Download controls
# !dx download -r path/to/controls/
# !dx download -r path/to/cases/

#!dx download 'ukb_rel.dat' -- this is related info for UKB -- use cohort browser to create

# Start by selecting the NDD you would like to use

## you will need to run the notebook multiple times to create one df for each NDD

In [ ]:
#Select NDD
ndd = 'DEM'

# Combine cases and controls

In [ ]:
#Load cases
#cases = pd.read_csv(f'AD_cases_filtered.txt', sep = '\t')
cases = pd.read_csv('dementia_cases_ALL.txt', sep = '\t')
#cases = pd.read_csv('MS_cases.txt', sep = '\t')
#cases = pd.read_csv('PD_cases_filtered_with_dementia.txt', sep = '\t')
#cases = pd.read_csv('PD_cases_strict_no_dementia.txt', sep = '\t')
#cases = pd.read_csv('ALS_cases.txt', sep = '\t')
#cases = pd.read_csv('vascular_dementia_cases.txt', sep = '\t')
cases

In [ ]:
# Rename columns to match my code
#cases = cases.rename(columns = {'eid':'ID', f'{ndd}_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
cases = cases.rename(columns = {'eid':'ID', f'Dementia_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
cases = cases[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
cases = cases.sort_values(by = f'{ndd}_DATE')
cases

# Because our medication data only runs until the end of 2017, we are selecting NDD cases from the same time period (up to 2018)

In [ ]:
#Select cases from BEFORE 2018
cases[['year', 'month', 'day']] = cases[f'{ndd}_DATE'].str.split('-', expand = True)
cases['year'] = cases['year'].astype(int)
cases = cases[cases['year'] < 2018]
cases = cases.sort_values(by = f'{ndd}_DATE')
cases = cases[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
cases = cases.drop_duplicates(subset = 'ID', keep = 'first')
cases

In [ ]:
#Load controls
controls = pd.read_csv('NDD_healthy_controls_current_age_60.txt', sep = '\t')
controls[f'{ndd}_DATE'] = np.nan
controls = controls.rename(columns = {'eid':'ID', f'{ndd}_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
controls = controls[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
controls = controls.drop_duplicates(subset = 'ID', keep = 'first')
controls

In [ ]:
# Combine cases and controls
df = pd.concat([cases, controls])

#Remove cases without TOWNEND 
df = df[~df['TOWNSEND'].isna()]

#Check to make sure no duplicate IDs
print(df.ID.value_counts())

df

In [ ]:
# Add BIRTH_YEAR
b = pd.read_csv('birth_death_participant.csv')
b = b.rename(columns = {'eid':'ID', 'p34':'BIRTH_YEAR', 'p40000_i0':'DATE_OF_DEATH'})
b = b[['ID', 'BIRTH_YEAR']]
b

In [ ]:
#Merge with cases/controls
df = df.merge(b, left_on = 'ID', right_on = 'ID', how = 'left')
df

In [ ]:
# Add DEATH YEAR IF BEFORE 2018
d = pd.read_csv('birth_death_participant.csv')
d = d.rename(columns = {'eid':'ID', 'p34':'BIRTH_YEAR', 'p40000_i0':'DATE_OF_DEATH'})
d = d[['ID', 'DATE_OF_DEATH']]
d = d[~d['DATE_OF_DEATH'].isna()]
d = d.sort_values(by = 'DATE_OF_DEATH')
d[['year', 'month', 'day']] = d.DATE_OF_DEATH.str.split('-', expand = True)
d['year'] = d['year'].astype(int)
d = d[d['year'] < 2018]
d = d[['ID', 'DATE_OF_DEATH']]
d

In [ ]:
#Merge with cases/controls
df = df.merge(d, left_on = 'ID', right_on = 'ID', how = 'left')
df

In [ ]:
#Check for NAs
print('Sex:', df.GENETIC_SEX.isna().value_counts())
print('Birth year:', df.BIRTH_YEAR.isna().value_counts())
print('AGE:', df.AGE_OF_RECRUIT.isna().value_counts())
print('TOWNSEND:', df.TOWNSEND.isna().value_counts())

In [ ]:
#Eliminate IDs with no birth year
df = df[~df['BIRTH_YEAR'].isna()]
df

In [ ]:
#Check number of cases and controls
df[f'{ndd}_DATE'].isna().value_counts()

# Add drugs codes

In [ ]:
# Load list of people for whom we have drug data
p = pd.read_csv('drugs_april_2024_participant.csv')
participant_list = list(p['eid'])
print(len(participant_list))

In [ ]:
# Only select people from the have drug data list
df = df[df['ID'].isin(participant_list)]
df

In [ ]:
df[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
# We came up with this list of drugs in notebook 03
drugs_list = ['moxifloxacin', 'fexofenadine', 'lormetazepam', 'nizatidine', 'lacidipine', 'rifaximin', 'levomepromazine', 'losartan', 'metformin', 'colchicine', 'valproicacid', 'acetylcysteine', 'phenoxymethylpenicillin', 'erdosteine', 'montelukast', 'sildenafil', 'amiodarone', 'clomethiazole', 'gemfibrozil', 'alogliptin', 'mirtazapine', 'aclidiniumbromide', 'clomipramine', 'nebivolol', 'minoxidil', 'clonidine', 'alfuzosin', 'rizatriptan', 'mecysteine', 'rimonabant', 'naratriptan', 'paracetamolandmetoclopramide', 'orlistat', 'griseofulvin', 'telmisartan', 'pravastatin', 'zaleplon', 'piroxicam', 'apixaban', 'amphotericin', 'linagliptin', 'amlodipine', 'diltiazem', 'nortriptyline', 'potassiumchlorideandpotassiumbicarbonate', 'propantheline', 'hydrotalcite', 'alprazolam', 'indomethacin', 'ciclesonide', 'nabumetone', 'perindopril', 'digoxin', 'oxerutins', 'amisulpride', 'bisacodyl', 'carbocisteine', 'gabapentin', 'misoprostol', 'ticagrelor', 'apomorphine', 'tetrabenazine', 'tetracycline', 'nalidixicacid', 'methylcellulose', 'flucloxacillin', 'epogam', 'mefenamic', 'dantrolene', 'cyproheptadine', 'phenylephrine', 'terbutaline', 'fentanyl', 'fluoxetine', 'moxisylyte', 'zonisamide', 'doxepin', 'sodiumvalproate', 'thioridazine', 'clomifene', 'fesoterodine', 'clonazepam', 'orciprenaline', 'fusidicacid', 'metronidazole', 'tramadol', 'ibandronicacid', 'gliclazide', 'naltrexone', 'chlortalidone', 'ciprofloxacin', 'orphenadrine', 'sulpiride', 'aminophylline', 'aspirin', 'glimepiride', 'budesonide', 'eprosartan', 'carvedilol', 'rupatadine', 'acemetacin', 'ibuprofen', 'aceclofenac', 'ketorolac', 'glyceryltrinitrate', 'olsalazine', 'zafirlukast', 'flurbiprofen', 'trimethoprim', 'cefixime', 'fosinopril', 'inositolnicotinate', 'entacapone', 'lercanidipine', 'acetazolamide', 'indapamide', 'metoprolol', 'febuxostat', 'captopril', 'disulfiram', 'propylthiouracil', 'tolbutamide', 'ezetimibe', 'risedronate', 'bisoprolol', 'pantoprazole', 'flavoxate', 'cinnarizine', 'nateglinide', 'hydrochlorothiazideandramipril', 'magnesium', 'eletriptan', 'trifluoperazine', 'chloramphenicol', 'alimemazine', 'moxonidine', 'oxcarbazepine', 'cetirizine', 'pseudoephedrine', 'albuterol', 'hydroxychloroquine', 'trimipramine', 'trimethoprimandsulfamethoxazole', 'nifedipine', 'sotalol', 'hyoscinebutylbromide', 'loratadine', 'erythromycin', 'naftidrofuryl', 'modafinil', 'esomeprazole', 'cisapride', 'oxytetracycline', 'tinidazole', 'sumatriptan', 'calcitriol', 'ranitidine', 'leflunomide', 'famciclovir', 'oxybutynin', 'perphenazine', 'ropinirole', 'ursodeoxycholicacid', 'citalopram', 'riluzole', 'darifenacin', 'buspirone', 'alendronatesodium', 'theophylline', 'paracetamolandcodeine', 'sulfasalazine', 'pre-exposureprophylaxis', 'lamotrigine', 'dexamethasone', 'stalevo', 'norfloxacin', 'ofloxacin', 'mianserin', 'nitrofurantoin', 'memantine', 'lorazepam', 'anticholinergic', 'midazolam', 'dronedarone', 'enalapril', 'fluticasone', 'lithium', 'mebendazole', 'diclofenac', 'quinapril', 'itraconazole', 'pyridostigmine', 'methylphenidate', 'trihexyphenidyl', 'co-amilofruse', 'mizolastine', 'irbesartan', 'glipizide', 'acamprosate', 'cyproterone', 'methotrexate', 'tolfenamicacid', 'prasugrel', 'mefloquine', 'methylprednisolone', 'isotretinoin', 'ibuprofenandcodeine', 'letrozole', 'nicardipine', 'magnesiumhydroxide', 'vancomycin', 'ipratropium', 'prednisone', 'potassiumchloride', 'saxagliptin', 'risperidone', 'dipipanone', 'tadalafil', 'astemizole', 'simvastatin', 'acarbose', 'quinine', 'levothyroxine', 'triamterene', 'nystatin', 'chlordiazepoxide', 'pramipexole', 'dihydrocodeine', 'allopurinol', 'terfenadine', 'mesalazine', 'hydrocortisonebuccaltablets', 'doxycycline', 'trandolapril', 'dicyclomine', 'asilone', 'nadolol', 'frovatriptan', 'ranolazine', 'azithromycin', 'cholestyramine', 'zopiclone', 'vigabatrin', 'tiotropium', 'bromocriptine', 'tiaprofenicacid', 'lidocaine', 'phenelzine', 'levofloxacin', 'diflunisal', 'hydralazine', 'acitretin', 'metoclopramideandaspirin', 'ciclosporin', 'methyldopa', 'tripotassiumdicitratobismuthate', 'galantamine', 'cyclobenzaprine', 'codeine', 'rifampicinandisoniazid', 'valaciclovir', 'phenytoin', 'co-amilozide', 'etoricoxib', 'sulindac', 'liothyronine', 'atenolol', 'cinacalcet', 'co-beneldopa', 'lansoprazole', 'indoramin', 'fenofibrate', 'zolmitriptan', 'hydrochlorothiazideandlisinopril', 'trospium', 'sucralfate', 'etamsylate', 'eplerenone', 'cromolynsodium', 'benzydamine', 'atorvastatin', 'cefalexin', 'minocycline', 'hydroxycarbamide', 'pioglitazone', 'rasagiline', 'carbamazepine', 'pivmecillinam', 'isosorbidedinitrate', 'formoterol', 'pentoxifylline', 'cabergoline', 'diazepam', 'furosemide', 'amiloride', 'tranexamicacid', 'ramipril', 'lofepramine', 'etidronicacid', 'promazine', 'aliskren', 'penicillamine', 'chlorpheniramine', 'prochlorperazine', 'bismuthsubsalicylate', 'cyclopenthiazide', 'finasteride', 'ironandfolicacid', 'penicillin', 'cyclizine', 'sertraline', 'ampicillin', 'dexketoprofen', 'nefazodone', 'solifenacin', 'felodipine', 'fluvoxamine', 'bupropion', 'tramacet', 'amoxicillin', 'tolterodine', 'isosorbidemononitrateandisosorbidedinitrate', 'betamethasone', 'cefradine', 'co-amoxiclav', 'clarithromycin', 'indometacin', 'almotriptan', 'escitalopram', 'chloroquine', 'ketotifen', 'oseltamivir', 'tamsulosin', 'olanzapine', 'levocetirizine', 'fluphenazine', 'chloralhydrate', 'tizanidine', 'cerivastatin', 'pholcodine', 'bezafibrate', 'methadone', 'bendroflumethiazide', 'methocarbamol', 'rivaroxaban', 'dosulepin', 'verapamil', 'mirabegron', 'salbutamol', 'amitriptyline', 'omeprazole', 'baclofen', 'fishoil', 'venlafaxine', 'meptazinol', 'cefpodoxime', 'fludrocortisone', 'levetiracetam', 'rosiglitazone', 'isoniazid', 'empagliflozin', 'carbimazole', 'lisinopril', 'doxazosin', 'co-magaldrox', 'temazepam', 'imipramine', 'gaviscon', 'mycophenolicacid', 'cefadroxil', 'methenamine', 'nicorandil', 'loperamide', 'mometasone', 'sterculia', 'cimetidine', 'hydroxyurea', 'celecoxib', 'meperidine', 'imidapril', 'terazosin', 'atropine', 'flecainide', 'thyroxine', 'hyoscinehydrobromide', 'dantronanddocusate', 'dabigatran', 'fluconazole', 'prucalopride', 'mebeverine', 'macrogol', 'lactulose', 'glibenclamide', 'sibutramine', 'meloxicam', 'valdecoxib', 'manevac', 'topiramate', 'ketoprofen', 'primidone', 'paroxetine', 'acrivastine', 'sodiumfusidate', 'strontium', 'zolpidem', 'oxprenolol', 'sulfinpyrazone', 'carisoprodol', 'pizotifen', 'clemastine', 'rivastigmine', 'bumetanide', 'agomelatine', 'famotidine', 'reboxetine', 'pregabalin', 'alprostadil', 'haloperidol', 'candesartan', 'kaolin', 'ivabradine', 'linaclotide', 'atenololandnifedipine', 'repaglinide', 'selegiline', 'olmesartan', 'oxazepam', 'paracetamol', 'dutasteride', 'prazosin', 'sodiumcromoglicate', 'amantadine', 'nefopam', 'mercaptopurine', 'insulin', 'docusate', 'buprenorphine', 'isosorbidemononitrate', 'chlorphenamine', 'diphenhydramine', 'valganciclovir', 'azathioprine', 'proguanil', 'propranolol', 'prednisolone', 'immunoglobin', 'metoclopramide', 'celiprolol', 'rofecoxib', 'trazodone', 'tamoxifen', 'raloxifene', 'moclobemide', 'metolazone', 'omega-3', 'dipyridamole', 'alverine', 'alendronicacid', 'quetiapine', 'duloxetine', 'simeticone', 'beclometasone', 'azapropazone', 'lymecycline', 'aripiprazole', 'domperidone', 'rabeprazole', 'hydrocortisone', 'phenobarbital', 'betahistine', 'salmeterol', 'hydroxyzine', 'loprazolam', 'prilocaine', 'triamcinolone', 'zoledronicacid', 'anastrozole', 'fybogel', 'canagliflozin', 'clopidogrel', 'balsalazide', 'vardenafil', 'lumiracoxib', 'labetalol', 'brompheniramine', 'danazol', 'clodronicacid', 'warfarin', 'methysergide', 'hydrochlorothiazide', 'sodiumpicosulfate', 'ondansetron', 'adalimumab', 'rifampicin', 'tenoxicam', 'colesevelam', 'ciprofibrate', 'chlorpromazine', 'pancrelipase', 'co-danthrusate', 'disopyramide', 'oxycodone', 'valsartan', 'nitrazepam', 'tacrolimus', 'lacosamide', 'flupentixol', 'cyclophosphamide', 'nitrousoxide', 'ethambutol', 'rosuvastatin', 'clobazam', 'donepezil', 'promethazine', 'spironolactone', 'co-careldopa', 'clindamycin', 'naproxen', 'senna', 'cefaclor', 'fluvastatin', 'propiverine', 'timolol', 'dutasterideandtamsulosin', 'dapagliflozin', 'migril', 'pilocarpine', 'aciclovir', 'morphine', 'sevelamer', 'etodolac', 'dapsone', 'sitagliptin', 'nedocromil', 'propafenone', 'oxitropium', 'cefuroxime']
print(len(drugs_list))

In [ ]:
# download the csv files of drug data if needed (see step 03)
#! dx download -r csv_files

In [ ]:
df

In [ ]:
# Add drug data to df
for drug in drugs_list:
    code = pd.read_csv(f'new_drug_csv/{drug}_with_date.csv')
    code = code[['ID', f'{drug}_DATE', f'{drug}_N']]
    df = df.merge(code, left_on = 'ID', right_on = 'ID', how = 'left')

In [ ]:
# Create list of headers drug + DATE
codes = []
for drug in drugs_list:
    a = drug + '_DATE'
    codes.append(a)
    
print(len(codes))

# Prep for COX

In [ ]:
#Set variables
STUDY_ENDS = '2018-01-01'

In [ ]:
#Drop duplicates
df = df.sort_values(by = f'{ndd}_DATE')
df = df.drop_duplicates(subset = 'ID', keep = 'first')
print(len(df))
df.head()

In [ ]:
#Calculate the year they were recruited to the study
df['recruit_year'] = df['BIRTH_YEAR']+df['AGE_OF_RECRUIT']

In [ ]:
#Create tenure for people with NDD - select folks with an NDD date
has_NDD = df[~df[ndd + '_DATE'].isna()]

#Calculate the tenure, i.e. the time from the beginning of the study to their time of NDD diagnosis
has_NDD['tenure'] = (pd.to_datetime(has_NDD[ndd + '_DATE'], errors = 'coerce') - pd.to_datetime(has_NDD['recruit_year'], format='%Y')).dt.days/365

#Only keep people who got NDD after they joined study
has_NDD = has_NDD[has_NDD['tenure'] > 0]

#Add age_at_tenure to people with an NDD
has_NDD['age_at_tenure'] = (pd.to_datetime(has_NDD[ndd + '_DATE'], errors = 'coerce') - pd.to_datetime(has_NDD['BIRTH_YEAR'], format='%Y')).dt.days/365

#Create tenure for people without NDD - select folks with no NDD date
NDD_free = df[df[ndd + '_DATE'].isna()]

#For people without NDD, break into dead and alive
alive = NDD_free[NDD_free['DATE_OF_DEATH'].isna()]
dead = NDD_free[~NDD_free['DATE_OF_DEATH'].isna()]

#Calculate the tenure for people who are still alive, i.e. the time from the beginning of the study to the end of study
alive['tenure'] = (pd.to_datetime(STUDY_ENDS) - pd.to_datetime(alive['recruit_year'], format='%Y')).dt.days/365

#Add age_at_tenure for people who are still alive
alive['age_at_tenure'] = (pd.to_datetime(STUDY_ENDS) - pd.to_datetime(alive['BIRTH_YEAR'], format='%Y')).dt.days/365

#Calculate the tenure for people who are still dead, i.e. the time from the beginning of the study to the end of study
dead['tenure'] = (pd.to_datetime(dead['DATE_OF_DEATH']) - pd.to_datetime(dead['recruit_year'], format='%Y')).dt.days/365

#Add age_at_tenure for people who are dead
dead['age_at_tenure'] = (pd.to_datetime(dead['DATE_OF_DEATH']) - pd.to_datetime(dead['BIRTH_YEAR'], format='%Y')).dt.days/365

#Combine two groups
df = pd.concat([has_NDD, alive, dead])

In [ ]:
#Encode NDD to 1 or 0
df[ndd] = np.where(df[ndd + '_DATE'].isna(), 0, 1)

#GENETIC_SEX to 1 or 2
df.loc[df.GENETIC_SEX == 'Female', 'GENETIC_SEX'] = '2'
df.loc[df.GENETIC_SEX == 'Male', 'GENETIC_SEX'] = '1'

# Add quartile info to the df

In [ ]:
new_drug_list = []
omit_drug_list = []
for code in drugs_list:
    df['Lag_' + code] = (pd.to_datetime(df[code + '_DATE'], errors = 'coerce') - pd.to_datetime(df['recruit_year'], format = '%Y')).dt.days/365
    
    quantiles = df[f'{code}_N'].quantile([0.25, 0.5, 0.75])
    #print(code)
    low = quantiles[0.25]
    #print(low)
    high = quantiles[0.75]
    #print(high)
    
    df['low_' + code] = np.where((df['Lag_' + code] < df['tenure']) & (df[code + '_N'] <= low), 1, 0)
    df['high_' + code] = np.where((df['Lag_' + code] < df['tenure']) & (df[code + '_N'] >= high), 1, 0)
    
    if high-low > 1:
        df['med_' + code] = np.where((df['Lag_' + code] < df['tenure']) & ((df[code + '_N'] > low) & (df[code + '_N'] < high)), 1, 0)
        new_drug_list.append(code)
    else:
        omit_drug_list.append(code)
        print(code)
        print(low)
        print(high)      

In [ ]:
print(len(omit_drug_list))
print(len(new_drug_list))
print(new_drug_list)

In [ ]:
#Find related individuals
related = pd.read_csv('ukb_rel.dat', sep = ' ')
related = related[related['Kinship']> 0.0884]
related

In [ ]:
#Create list of all people in our df with NDD status for analysis
test = df[['ID', f'{ndd}']]
test

In [ ]:
#Create list of all people in our df
ndd_ids = list(df['ID'])
#ndd_ids = list(cases['ID'])

#Create list of people who are related & in our df
remove = related.loc[related['ID1'].isin(ndd_ids) & related['ID2'].isin(ndd_ids)]
remove

In [ ]:
remove = remove.merge(test, left_on = 'ID1', right_on = 'ID', how = 'left')
remove = remove.rename(columns = {f'{ndd}':f'{ndd}1'})
remove = remove.merge(test, left_on = 'ID2', right_on = 'ID', how = 'left')
remove = remove.rename(columns = {f'{ndd}':f'{ndd}2'})
remove

In [ ]:
# test = remove[remove['PD2'] == 1]
# test

In [ ]:
#This prioritizes people without an NDD
remove_list = []
for i in range (len(remove)):
    if remove[f'{ndd}1'][i] == 0:
        remove_list.append(remove['ID1'][i])
    else:
        remove_list.append(remove['ID2'][i])

In [ ]:
print(len(remove_list))

In [ ]:
df_final = df[~df['ID'].isin(remove_list)]
df_final

In [ ]:
removed_df = df[df['ID'].isin(remove_list)]
removed_df[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
df_final[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
df_final

In [ ]:
df_final.to_csv(f'{ndd}_with_tenure_low_med_high_NOV_12_24.csv', header = True, index = None)

In [ ]:
print(f'!dx upload {ndd}_with_tenure_low_med_high_NOV_12_24.csv --path /data/files_for_cox/{ndd}_with_tenure_low_med_high_NOV_12_24.csv')

# Add in ICD10 codes

In [ ]:
! dx download -r data/UPDATED_ICD10_dates/

In [ ]:
#list of icd10_codes
# Note we removed a few from the previous list because they were removed from controls, had less than 20 individuals, or data was not available in AoU
icd10_list = ['A04', 'B02', 'B37', 'E10', 'E11', 'E27', 'E78', 'E87', 'F20', 'F31', 'F32', 'F33', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'G40', 'G43', 'G47', 'H66', 'I10', 'I11', 'I12', 'I15', 'I20', 'I21', 'I25', 'I47', 'I48', 'I49', 'I50', 'I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I69', 'I82', 'K04', 'K05', 'K20', 'K21', 'K22', 'K25', 'K31', 'K51', 'K59', 'K70', 'K71', 'K72', 'K73', 'K74', 'K75', 'K76', 'L25', 'L40', 'L50', 'M06', 'M13', 'M15', 'M16', 'M17', 'M18', 'M19', 'M32', 'M45', 'M79', 'M80', 'M81', 'M88', 'N04', 'N10', 'N18', 'N19', 'N30', 'N31', 'N32', 'N39', 'N40', 'N94']
print(len(icd10_list))
print(icd10_list)

In [ ]:
# Here we remove codes that don't have a date or occur before 1999 or after 2018 -- this is to match our medication data.
for code in icd10_list:
    print(code)
    df = pd.read_csv(f'UPDATED_ICD10_dates/{code}_with_date.csv')
    #df = pd.read_csv(f'{code}_with_date.csv')
    print(len(df))
    df = df[df[code] != "Code has event date matching participant's date of birth"]
    df = df[df[code] != "Code has event date after participant's date of birth and falls in the same calendar year as date of birth"]
    df = df[df[code] != "Code has event date in the future and is presumed to be a place-holder or other system default {2037-07-07}"]
    df['year'] = df[f'{code}'].str.split('-', expand=True)[0]
    df['year'] = df['year'].astype(int)
    df = df[df['year'] < 2018]
    df = df[df['year'] >= 1999]
    df = df[['ID', f'{code}']]
    print(len(df))
    df.to_csv(f'{code}_with_date_1999.csv', header = True, index = False)

In [ ]:
# Look at one file
test = pd.read_csv(f'M81_with_date_1999.csv')
test

# Add covariates to original dataframe

In [ ]:
# Load df -- need to run notebook for all NDDs: AD, PD, and DEM
ndd = 'AD'
df = pd.read_csv(f'{ndd}_with_tenure_low_med_high_NOV_12_24.csv', low_memory = False)
df

In [ ]:
print(len(icd10_list))

In [ ]:
# Add ICD10 data to the orignial df
for code in icd10_list:
    c = pd.read_csv(f'{code}_with_date_1999.csv')
    df = df.merge(c, left_on = 'ID', right_on = 'ID', how = 'left')
    print(len(df))

In [ ]:
# Check to see how many ICD10 values we had for each code
for code in icd10_list:
    print(code)
    print(df[f'{code}'].isna().value_counts())

In [ ]:
#We only want to include an ICD10 code as a covariate if it was recorded BEFORE the end of the study
for code in icd10_list:
    df['Lag_' + code] = (pd.to_datetime(df[code], errors = 'coerce') - pd.to_datetime(df['recruit_year'], format = '%Y')).dt.days/365
        
    #Select data if it happened before study end -- lag 0
    df['QC0_' + code] = np.where((df['Lag_' + code] < df['tenure']), 1, 0)

In [ ]:
# Check to make sure we have at least 20 samples for each ICD10 code
remove_list = []
for n in icd10_list:
    test = df[df[f'QC0_{n}']==1]
    print(n)
    print(len(test))
    if len(test) < 20:
        remove_list.append(n)
    else:
        pass

In [ ]:
remove_list

# Add APOE status

In [ ]:
#! dx download apoe/apoe_snps_genotypes.APOE_GENOTYPES.csv

In [ ]:
apoe = pd.read_csv('apoe_snps_genotypes.APOE_GENOTYPES.csv')
apoe = apoe[['IID', 'APOE_GENOTYPE']]
apoe

In [ ]:
df = df.merge(apoe, left_on = 'ID', right_on = 'IID', how = 'left')
df

In [ ]:
df.APOE_GENOTYPE.value_counts(dropna = False)

In [ ]:
# One-hot encode the 'APOE_GENOTYPE' column
df_encoded = pd.get_dummies(df, columns=['APOE_GENOTYPE'], prefix='', prefix_sep='')

# Display the result
df_encoded.columns

In [ ]:
list_columns = list(df.columns)
cleaned_list1 = [x for x in list_columns if not x.startswith('Lag')]
cleaned_list2 = [x for x in cleaned_list1 if not x.endswith('DATE')]
print(len(cleaned_list2))
#print(cleaned_list2)

In [ ]:
df2 = df_encoded[cleaned_list2]
df2

In [ ]:
df2

In [ ]:
df2.to_csv(f'{ndd}_low_med_high_and_ICD10_APOE_MAY_14_2025.csv', header = True, index=False)

In [ ]:
!dx upload {ndd}_low_med_high_and_ICD10_APOE_MAY_14_2025.csv --path /data/files_for_cox/{ndd}_low_med_high_and_ICD10_APOE_MAY_14_2025.csv